In [2]:
from pyspark.sql import SparkSession
import os
from datetime import datetime, timedelta

# Create a Spark session
spark = SparkSession.builder \
    .appName("SedonaApp") \
    .config("spark.jars.packages", "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.0-incubating") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryo.registrator", "org.apache.sedona.core.serde.SedonaKryoRegistrator") \
    .getOrCreate()

# Set Sedona configuration
spark.sparkContext.setSystemProperty("sedona.global.charset", "utf8")

:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/maxheilingbrunner/.ivy2/cache
The jars for the packages stored in: /Users/maxheilingbrunner/.ivy2/jars
org.apache.sedona#sedona-python-adapter-3.0_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f206d3ef-e076-4f65-8aff-06a2a7fc1086;1.0
	confs: [default]
	found org.apache.sedona#sedona-python-adapter-3.0_2.12;1.0.0-incubating in central
	found org.locationtech.jts#jts-core;1.18.0 in central
	found org.wololo#jts2geojson;0.14.3 in central
	found org.apache.sedona#sedona-core-3.0_2.12;1.0.0-incubating in central
	found org.apache.sedona#sedona-sql-3.0_2.12;1.0.0-incubating in central
downloading https://repo1.maven.org/maven2/org/apache/sedona/sedona-python-adapter-3.0_2.12/1.0.0-incubating/sedona-python-adapter-3.0_2.12-1.0.0-incubating.jar ...
	[SUCCESSFUL ] org.apache.sedona#sedona-python-adapter-3.0_2.12;1.0.0-incubating!sedona-python-adapter-3.0_2.12.jar (1645ms)
downloading https://repo1.maven.org/maven2/o

In [3]:
from sedona.register import SedonaRegistrator

SedonaRegistrator.registerAll(spark)

ModuleNotFoundError: No module named 'sedona.register'

In [ ]:
from sedona.utils.adapter import Adapter
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.SpatialRDD import SpatialRDD

# Read a shapefile into a SpatialRDD
spatial_rdd = ShapefileReader.readToGeometryRDD(spark.sparkContext, "path_to_shapefile")

# Transform the SpatialRDD to a DataFrame
spatial_df = Adapter.toDf(spatial_rdd, spark)

# Register the DataFrame as a temporary table and run SQL queries
spatial_df.createOrReplaceTempView("spatial_table")
result = spark.sql("SELECT * FROM spatial_table WHERE st_contains(geometry, st_point(1.5, 1.5))")

# Show the results
result.show()

In [10]:
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import os

# Initialize Spark
spark = SparkSession.builder.appName("ConcatCSVFiles").getOrCreate()

# Folder where the CSV files are located
folder_path = "/Users/maxheilingbrunner/Documents/solar-energy-spatial-prediction/data"

# Define the range of months
start_month = datetime.strptime("202203", "%Y%m")
end_month = datetime.strptime("202302", "%Y%m")

# Initialize an empty DataFrame to store final concatenated data
malaga_df = None

# Loop through each month in the range
current_month = start_month
while current_month.year < end_month.year or (current_month.year == end_month.year and current_month.month <= end_month.month):
    # Construct file names
    file_suffix = current_month.strftime("%Y%m")
    file_electricity = os.path.join(folder_path, f"{file_suffix}_ELECTRICIDAD_MALAGA.csv")
    
    # Read the CSV files into Spark DataFrames
    if os.path.exists(file_electricity):
        electricity_df = spark.read \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .csv(file_electricity)
        
        # Concatenate the individual DataFrame with the final DataFrame
        if malaga_df is None:
            malaga_df = electricity_df
        else:
            malaga_df = malaga_df.union(electricity_df)
        
    else:
        print(f"The files for {file_suffix} were not found in the folder '{folder_path}'.")

    # Move to the next month
    month, year = (current_month.month + 1, current_month.year) if current_month.month < 12 else (1, current_month.year + 1)
    current_month = datetime(year, month, 1)

# Now final_df contains the concatenated data. You can perform further operations on it.
if malaga_df:
    malaga_df.show()
else:
    print("No data to process.")

In [ ]:
spark.stop()